In [1]:
library(dplyr)
library(magrittr)
library(lubridate)
library(stringr)
library(reshape2)

mimic_db <- src_postgres(dbname="mimic",port=5432,user="postgres",host="localhost")


Attaching package: ‘dplyr’

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: RPostgreSQL
Warning message:
: package ‘RPostgreSQL’ was built under R version 3.1.3Loading required package: DBI


I use the following command to convert the default date format of MIMIC

ALTER TABLE admissions ALTER COLUMN edouttime TYPE TIMESTAMP WITH TIME ZONE USING edouttime AT TIME ZONE 'UTC';

# View all Admissions and Filter out Non-Trauma

In [2]:
#these are all pointers to POSTGRES they have not been "collected" yet
diagnosesIcd <- tbl(mimic_db,"diagnoses_icd") %>%
    filter(icd9_code %like% 'E%') %>%
    select(subject_id,hadm_id,icd9_code)

demographics <- tbl(mimic_db,"patients") %>%
    select(subject_id,gender,dob)

admissions <- tbl(mimic_db, "admissions") %>%
    select(subject_id,hadm_id,admittime,dischtime,edregtime,edouttime,deathtime,admission_type,admission_location,diagnosis) %>%
    inner_join(diagnosesIcd) %>%
    inner_join(demographics)

Joining by: c("subject_id", "hadm_id")
Joining by: "subject_id"


In [9]:
mv <- '([E][8][12][0-9][0-9])'
water <- '([E][8][3][0-9][0-9])'
otherVehicle <- '([E][8][4][^9][0-9])'
falls <- '([E][8][8][0-9][0-9])'
burn <- '([E][8][9][0-9][0-9])'
penetrated <- '([E][9][1][45][0-9])'
other1 <- '([E][9][1][6789][0-9])'
other2 <- '([E][9][2][12345][0-9])'
homicide <- '([E][9][6][0-9][0-9])'
suicide <- '([E][9][5][567][0-9])'
unknown <- '([E][9][8][0-9][0-9])'

traumaIcd9 <- paste(mv,water,otherVehicle,falls,burn,penetrated,other1,
                    other2,homicide,suicide,unknown,sep='|')

#dplyr automatically converts date strings to date objects when we collect
#we need to convert back to string to store back in to the data base
trAdmissionsTemp1 <- admissions %>%
    collect %>%
    filter(grepl(traumaIcd9,icd9_code)) %>%
    mutate_each("as.character",admittime,dischtime,edregtime,edouttime,deathtime,dob) %>%
    
    filter(admission_location == "EMERGENCY ROOM ADMIT") %>%

    #we need to convert to string for lubridate to work then back again to put back into postgres
    mutate(admittime=ymd_hms(admittime),dob=ymd(dob),age=year(admittime)-year(dob)) %>%
    mutate_each("as.character",admittime,dob) %>%
    select(-dob)

In [34]:
admitDischTimes <- trAdmissionsTemp1 %>%
    select(subject_id,hadm_id,admittime,dischtime) %>%
    mutate_each(funs(ymd_hms),admittime,dischtime)

save(admitDischTimes,file="/Users/Arya/Petzold/Coagulopathy/mimicPatientView/data/admitDischTimes.Rdata")

In [35]:
#convert absolute times to relative times
start <- "2000-01-01 00:00:00" %>% ymd_hms
trAdmissionsTemp2 <- trAdmissionsTemp1 %>%
    mutate_each(funs(ymd_hms),admittime,dischtime,edregtime,edouttime,deathtime) %>%
    mutate(hoursToAdmit=round(as.numeric(difftime(admittime,edregtime,units="hours")),digits=1)) %>%
    mutate(daysInHospital=ceiling(as.numeric(difftime(dischtime,admittime,units="days")))) %>%
    mutate(deathtime=ymd_hms(deathtime)) %>%
    mutate(admittime=ymd_hms(admittime)) %>%
    mutate(deathTime=start+difftime(deathtime,admittime,units="secs")) %>%
    select(subject_id,hadm_id,gender,age,admission_location,diagnosis,icd9_code,hoursToAdmit,daysInHospital,deathTime)

# Copy the Keys of Trauma Incidences Back to DB and to Rdata

Don't copy admissions back to DB until we augment it with other important info such as number of blood units recieved

In [59]:
#if we get trAdmissions table already exists in postgres database we
#can delete it with 'DROP TABLE IF EXISTS "trAdmissions";'
#mimiciii needs to be in the search path first though with
#'SET search_path=test,public,mimiciii;'

#we include the registration time in to the ER to compare everyone
#on the same time scale
trKeys <- trAdmissionsTemp1 %>%
    select(subject_id,hadm_id,edregtime,admittime)

trKeysPointer <- copy_to(mimic_db,trKeys,temporary=FALSE)

# Augment Admissions with Notes and Input Info

We'd like to add the following:
* injury category (e.g. fall MVC etc.)
* injury severity
* units of RBC, plt, and ffp recieved

## Notes Wrangling

### What type of Notes Do we Have?

In [12]:
print("Unique Patients: ")
tbl(mimic_db,"trKeys") %>% collect %>% nrow

[1] "Unique Patients: "


[1] 3734

In [13]:
notesSummary <- tbl(mimic_db,"noteevents") %>%
    inner_join(trKeysPointer) %>%
    group_by(category,subject_id) %>%
    summarise(count=n()) %>%
    group_by(category) %>%
    collect %>%
    summarise(uniquePatients=length(unique(subject_id)), medNumNotesPerPatient=median(count))

Joining by: c("subject_id", "hadm_id")


In [14]:
notesSummary

,category,uniquePatients,medNumNotesPerPatient
1,Case Management,29,1
2,Consult,6,1
3,Discharge summary,3323,1
4,ECG,2410,2
5,Echo,878,1
6,General,226,2
7,Nursing,897,13
8,Nursing/other,2518,6
9,Nutrition,300,2
10,Pharmacy,3,1


### Wrangle Discharge Notes

## Wrangle Blood Product Inputs

In [15]:
dItemsPointer <- tbl(mimic_db,"d_items")

trInputCv <- tbl(mimic_db,"inputevents_cv") %>%
    inner_join(trKeysPointer) %>%
    select(subject_id,hadm_id,admittime,itemid,charttime,amount,amountuom,rate,rateuom) %>%
    inner_join(dItemsPointer) %>%
    select(subject_id,hadm_id,admittime,itemid,label,abbreviation,unitname,category,charttime,amount,amountuom,rate,rateuom) %>%
    collect %>%
    mutate_each("as.character",charttime,admittime)

Joining by: c("subject_id", "hadm_id")
Joining by: "itemid"


In [16]:
trInputMv <- tbl(mimic_db,"inputevents_mv") %>%
    inner_join(trKeysPointer) %>%
    select(subject_id,hadm_id,admittime,itemid,starttime,endtime,amount,amountuom,rate,rateuom) %>%
    inner_join(dItemsPointer) %>%
    select(subject_id,hadm_id,admittime,itemid,label,abbreviation,unitname,category,starttime,endtime,amount,amountuom,rate,rateuom) %>%
    collect %>%
    mutate_each("as.character",starttime,endtime,admittime)

Joining by: c("subject_id", "hadm_id")
Joining by: "itemid"


### Consolidate Notes from MV and CV Into a Single Table (MV is newer and does starttime and endtime instead of just rate)

for now we will just use charttime as starttime and not worry about end time or rate

In [17]:
trInputCvTemp <- trInputCv %>%
    #mutate(charttime=parse_date_time(charttime,"%y-%m-%d %H:%M:%S")) %>%
    select(subject_id,hadm_id,admittime,itemid,label,charttime,amount,amountuom)

trInput <- trInputMv %>%
    mutate(charttime=starttime) %>%
    select(subject_id,hadm_id,admittime,itemid,label,charttime,amount,amountuom) %>%
    rbind(trInputCvTemp) %>%
    mutate(edregtime=ymd_hms(admittime)) %>%
    mutate(charttime=ymd_hms(charttime))

In [36]:
bloodProducts <- ignore.case("RBC|FFP|Platelet")
start <- "2000-01-01 00:00:00" %>% ymd_hms

trBloodProducts <- trInput %>%
    filter(str_detect(label,bloodProducts)) %>%
    mutate(label=ifelse(str_detect(label,"RBC"),"RBC",
                         ifelse(str_detect(label,"FFP"),"FFP","PLT"))) %>%
    rename(value=amount) %>%
    rename(valueuom=amountuom) %>%
    mutate(valuenum=value) %>%
    mutate(timeSinceAdmit=start+difftime(charttime,admittime,units="secs")) %>%
    mutate(measType="Transfusion") %>%
    select(subject_id,hadm_id,itemid,label,timeSinceAdmit,value,valuenum,valueuom,measType)

### Save as Rdata

In [37]:
save(trBloodProducts,file="/Users/Arya/Petzold/Coagulopathy/mimicPatientView/data/trBloodProducts.Rdata")

### Augment Admissions

In [19]:
trBloodProductsSummary <- trBloodProducts %>%
    #change NAs to zero, after summary we want NA to represent that the
    #patient did not recieve that transfusion NOT that they did recieve it
    #it but that it was recorded NA
    mutate(valuenum=ifelse(is.na(valuenum),0,valuenum)) %>%
    group_by(subject_id,hadm_id,label) %>%
    summarise(totalRecordedMg=sum(valuenum,na.rm=TRUE)) %>%
    dcast(subject_id+hadm_id~label) %>%
    rename(totalFFP=FFP,totalPLT=PLT,totalRBC=RBC)

Using totalRecordedMg as value column: use value.var to override.


In [29]:
trPatientsTemp2 <- trAdmissionsTemp2 %>%
    left_join(trBloodProductsSummary) %>%
    select(subject_id,hadm_id,gender,age,admission_location,diagnosis,icd9_code,totalRBC,totalPLT,totalFFP,
          hoursToAdmit,daysInHospital,deathTime)

Joining by: c("subject_id", "hadm_id")


## Save Augmented Admissions Back to DB and as Rdata

In [30]:
trPatients <- trPatientsTemp2
save(trPatients,file="/Users/Arya/Petzold/Coagulopathy/mimicPatientView/data/trPatients.Rdata")

# Save Chartevents, Labevents, and Output Events For Trauma Patients

## Chartevents

chart events are really large so I wrangle them in the file rePartitionChartEvents.R

## Labevents

In [38]:
dLabItemsPointer <- tbl(mimic_db,"d_labitems")

trLabEvents <- tbl(mimic_db,"labevents") %>%
    inner_join(trKeysPointer) %>%
    select(subject_id,hadm_id,itemid,charttime,admittime,value,valuenum,valueuom) %>%
    left_join(dItemsPointer) %>%
    select(subject_id,hadm_id,itemid,label,charttime,admittime,value,valuenum,valueuom) %>%
    collect %>%
    mutate_each("as.character",charttime,admittime) %>%
    mutate(charttime=ymd_hms(charttime)) %>%
    mutate(edregtime=ymd_hms(admittime)) %>%
    mutate(timeSinceAdmit=start+difftime(charttime,admittime,units="secs")) %>%
    mutate(measType="Lab") %>%
    select(subject_id,hadm_id,itemid,label,timeSinceAdmit,value,valuenum,valueuom,measType)

Joining by: c("subject_id", "hadm_id")
Joining by: "itemid"
Warning message:
: All formats failed to parse. No formats found.

In [39]:
save(trLabEvents,file="/Users/Arya/Petzold/Coagulopathy/mimicPatientView/data/trLabEvents.Rdata")

## Combine Chart and Lab Events

## Output events

# Save Transfers for Trauma Patients

In [64]:
library(lubridate)

start <- "2000-01-01 00:00:00" %>% ymd_hms

trTransfersEdReg <- trKeysPointer %>%
    collect %>%
    mutate(edregtime=ymd_hms(edregtime)) %>%
    mutate(admittime=ymd_hms(admittime)) %>%
    mutate(timeSinceAdmit=start+difftime(edregtime,admittime,units="secs")) %>%

    mutate(itemid=as.integer(NA),label="Transfer",value="ED",
           valuenum=as.double(subject_id),valueuom="",measType="Transfer",fluid=NA,isFactor=TRUE) %>%
    select(subject_id,hadm_id,itemid,label,timeSinceAdmit,value,valuenum,valueuom,measType,)

trTransfersTemp <- tbl(mimic_db,"transfers") %>%
    inner_join(trKeysPointer) %>%
    select(subject_id,hadm_id,eventtype,curr_careunit,edregtime,admittime,intime) %>%
    
    mutate(intimeChr=as.character(intime)) %>%
    collect %>%
    mutate(intime=intimeChr) %>%
    select(-intimeChr) %>%

    mutate(intime=ymd_hms(intime)) %>%
    mutate(admittime=ymd_hms(admittime)) %>%
    mutate(timeSinceAdmit=start+difftime(intime,admittime,units="secs")) %>%

    mutate(itemid=as.integer(NA),label="Transfer",value=curr_careunit,
           valuenum=as.double(subject_id),valueuom="",measType="Transfer",fluid=NA,isFactor=TRUE) %>%
    select(subject_id,hadm_id,itemid,label,timeSinceAdmit,value,valuenum,valueuom,measType,
          fluid,isFactor)

trTransfers <- rbind(trTransfersEdReg,trTransfersTemp)

save(trTransfers,file="/Users/Arya/Petzold/Coagulopathy/mimicPatientView/data/trTransfers.Rdata")

Joining by: c("subject_id", "hadm_id")


In [50]:
trTransfers

ERROR: Error in as.POSIXlt.character(x, tz, ...): character string is not in a standard unambiguous format


Source: local data frame [15,151 x 6]

   subject_id hadm_id eventtype curr_careunit           admittime     intime
1          43  146828     admit         TSICU 2186-10-01 16:15:00 2186-10-01
2          43  146828  transfer               2186-10-01 16:15:00 2186-10-05
3          43  146828  transfer         TSICU 2186-10-01 16:15:00 2186-10-05
4          43  146828  transfer               2186-10-01 16:15:00 2186-10-06
5          43  146828 discharge               2186-10-01 16:15:00 2186-10-11
6          45  157907     admit         TSICU 2129-06-09 17:32:00 2129-06-10
7          45  157907  transfer               2129-06-09 17:32:00 2129-06-11
8          45  157907  transfer               2129-06-09 17:32:00 2129-06-14
9          45  157907  transfer               2129-06-09 17:32:00 2129-06-14
10         45  157907  transfer               2129-06-09 17:32:00 2129-06-14
..        ...     ...       ...           ...                 ...        ...

In [47]:
trTransfers %>% ggplot(aes(curr_careunit)) + geom_bar()

ERROR: Error in library(ggplot2): there is no package called ‘ggplot2’


ERROR: Error in eval(expr, envir, enclos): could not find function "ggplot"


## How many patients recieved blood?

In [ ]:
rbcItems <- tbl(mimic_db,"d_items")  %>%
    collect %>%
    filter(grepl("RBC",label))

In [ ]:
rbcItems

In [ ]:
recievedRbcCv <- tbl(mimic_db,"inputevents_cv") %>%
    filter(itemid %in% rbcItems$itemid) %>%
    select(subject_id) %>%
    collect

recievedRbcMv <- tbl(mimic_db,"inputevents_mv") %>%
    filter(itemid %in% rbcItems$itemid) %>%
    select(subject_id) %>%
    collect

In [ ]:
recievedRbc <- c(unique(recievedRbcCv$subject_id),unique(recievedRbcMv$subject_id))

In [ ]:
traumaAdmissions <- traumaAdmissions %>%
    mutate(recievedRbc = subject_id %in% recievedRbc)

In [ ]:
traumaAdmissions %>%
    mutate(died=!is.na(deathtime)) %>%
    group_by(recievedRbc) %>%
    summarize(total=n(),nDead=sum(died))

In [ ]:
library(dplyr)
load(file="../data/traumaAd.Rdata")

## Don't Rerun Code Past this point!

In [ ]:
traumaPatientChartEvents <- tbl(mimic_db,"chartevents") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    collect

In [ ]:
traumaPatientChartEventsatientChartEventsatientChartEvents

In [ ]:
traumaPatientChartEvents <- tbl(mimic_db,"chartevents") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    compute(name="traumaPatientChartEvents")

In [ ]:
traumaPatientChartEventsLocal <- traumaPatientChartEvents %>% collect

In [ ]:
save(traumaPatientChartEventsLocal,file="../data/traumaChartEvents.Rdata")

In [ ]:
transfers <- tbl(mimic_db,"transfers") %>%
    
filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,intime,outtime)

In [ ]:
transfers %>% select(hadm_id,icustay_id,dbsource,eventtype,intime,outtime)

In [ ]:
notes <- tbl(mimic_db,"noteevents") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,chartdate,category,description,text) %>%
    collect

In [ ]:
notes %>% print.data.frame

In [ ]:
d_items <- tbl(mimic_db,"d_items") %>% collect
charts <- tbl(mimic_db,"chartevents") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    collect %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,value,valuenum,valueuom)

In [ ]:
d_items <- tbl(mimic_db,"d_items") %>% collect
charts <- tbl(mimic_db,"chartevents") %>%
    filter(subject_id %in% traumaAdmissions$subject_id) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    collect
    #left_join(d_items) %>%
    #select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,value,valuenum,valueuom)

In [ ]:
charts %>% select(itemid,label,value,valuenum,charttime) %>% arrange(charttime) %>% print.data.frame

In [ ]:
charts %>% group_by(itemid,label) %>% summarize(count=n()) %>% ungroup() %>% arrange(desc(count)) %>% print.data.frame

In [ ]:
chartIDs <- c(211, #heart rate
             549,8456 #post tib pule Right/Left
             617,  #resp pattern
             618,  #resp rate
             646,  #SpO2
             455,8441 #systolic/diastolic
             678,  #temp.
             467,  #breathing device
             470,  #O2 flow
             524,  #pain level
             1087, #pain assessment type
             525,  #pain location
             522,  #pain cause
             527,  #pain type
             31,   #activity
             32,   #activity tolerance
             1337, #Riker-SAS
             425,428,593,599,#LLL,LUL,RLL,RUL
             80,   #bowel sounds,
             156,8381,  #Dorsal PedPulse Right/Left
             184,432,454,723,  #eye opening, level of conciousness, motor response, verbal response)
             704, #turn
             1529, #glucose
             77, #behavior
             781, #BUN
             786, #calcium
             787, #CO2
             788, #chloride
             791, #creatinine
             813, #hematocrit
             814, #hemoglobin
             821, #magnesium
             827, #phosporus
             828, #platelets
             829, #potassium
             833, #RBC
             837, #sodium
             861 #WBC)
injuryType <- c(344,352,356,362,374,478,644,8406)
charts %>% filter

In [ ]:
d_labitems <- tbl(mimic_db,"d_labitems") %>% collect
labevents <- tbl(mimic_db,"labevents") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    collect %>%
    left_join(d_labitems) %>%
    select(subject_id,hadm_id,itemid,label,fluid,category,charttime,value,valuenum,valueuom)

## Trauma Lab Events

In [ ]:
d_labitems <- tbl(mimic_db,"d_labitems")
labevents <- tbl(mimic_db,"labevents") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    left_join(d_labitems) %>%
    select(subject_id,hadm_id,itemid,label,fluid,category,charttime,value,valuenum,valueuom) %>%
    collect

save(labevents,file="../data/traumaLabEvents.Rdata")

In [ ]:
labevents %>% arrange(charttime) %>% print.data.frame

## Input Save

In [ ]:
input <- tbl(mimic_db,"inputevents_cv") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,itemid,charttime,amount,amountuom,rate,rateuom) %>%
    collect %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,amount,amountuom,rate,rateuom)

In [ ]:
d_items <- tbl(mimic_db,"d_items")
trInputCv <- tbl(mimic_db,"inputevents_cv") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,charttime,amount,amountuom,rate,rateuom) %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,amount,amountuom,rate,rateuom) %>%
    collect

save(trInputCv,file="../data/traumaInputEventsCv.Rdata")

In [ ]:
d_items <- tbl(mimic_db,"d_items")
trInputMv <- tbl(mimic_db,"inputevents_mv") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,starttime,endtime,amount,amountuom,rate,rateuom) %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,starttime,endtime,amount,amountuom,rate,rateuom) %>%
    collect

save(trInputMv,file="../data/traumaInputEventsMv.Rdata")

In [ ]:
trInputCv$charttime

## Output

In [ ]:
d_items <- tbl(mimic_db,"d_items")
trOutput <- tbl(mimic_db,"outputevents") %>%
    semi_join(trAdKeysDb) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valueuom) %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,value,valueuom) %>%
    collect

save(trInput,file="../data/traumaOutputEvents.Rdata")

## 28778 	157124

In [ ]:
transfers <- tbl(mimic_db,"transfers") %>%
    filter(subject_id == 28778,hadm_id==157124) %>%
    select(subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,intime,outtime)

In [ ]:
transfers %>% select(hadm_id,icustay_id,prev_careunit,dbsource,eventtype,intime,outtime)

In [ ]:
notes <- tbl(mimic_db,"noteevents") %>%
    filter(subject_id == 28778,hadm_id==157124) %>%
    select(subject_id,hadm_id,chartdate,category,description,text) %>%
    collect

In [ ]:
notes %>% print.data.frame

In [ ]:
d_items <- tbl(mimic_db,"d_items") %>% collect
charts <- tbl(mimic_db,"chartevents") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom) %>%
    collect %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,value,valuenum,valueuom)

In [ ]:
charts %>% select(itemid,label,value,valuenum,charttime) %>% arrange(charttime) %>% print.data.frame

In [ ]:
input <- tbl(mimic_db,"inputevents_cv") %>%
    filter(subject_id == 14427,hadm_id==167648) %>%
    select(subject_id,hadm_id,itemid,charttime,amount,amountuom,rate,rateuom) %>%
    collect %>%
    left_join(d_items) %>%
    select(subject_id,hadm_id,itemid,label,abbreviation,unitname,category,charttime,amount,amountuom,rate,rateuom)